In [1]:
# initiation
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
import seaborn as sns
import constrNMPy
import os

plt.rcParams['font.sans-serif'] = ['Times New Roman'] # set the font inline to Times New Roman

# os.getcwd() 
os.chdir('/Users/abel/Desktop/Junior/CogMod/Assignments/H2') # change the working directory to the folder where the data is stored

# color pallette
[g,y,r,b,p,i,k] = ['#8ECFC9', '#FFBE7A', '#FA7F6F', '#82B0D2','#BEB8DC', '#E7DAD2','#999999']

现有两枚骰子dice 1和dice 2，各有6个面（1, 2, 3, 4, 5, 6)，未知掷出各个面的概率是否均匀。对两枚骰子掷出不同点数的概率，有以下2种假设：
+ H1：两枚骰子相互独立，dice 1有$a$的概率掷出1，$(1 - a)/5$的概率掷出其他结果，dice 2有$b$的概率掷出6，$(1 - b)/5$的概率掷出其他结果，其中$a,\,b \in (0,1)$；
+ H2：两枚骰子不独立，有$c$的概率dice 1掷出1且dice 2掷出6，有$c$的概率dice 1掷出6且dice 2掷出1，有$(1 - 2c)/34$的概率掷出除一枚掷出1且另一枚掷出6之外的任意一种组合，其中$c \in (0,0.5)$。

### Q1
假设真实情况符合H1，$a = b = 1/3$。请固定随机种子为1，生成2组掷骰子的数据，第一组数据包含将2枚骰子一起投掷20次的结果，第二组数据包含将2枚骰子一起投掷1000次的结果。分别在两组数据上拟合H1和H2对应的模型，并比较AIC，报告最优的模型。

### Q2
比较在两组数据上的拟合结果，简述数据量对拟合结果中参数的影响，以及对模型比较的影响。

### Q3
分别假设骰子的真实情况符合H1和H2，当真实情况为H1时$a=b=1/3$，当真实情况为H2时$c=1/15$。分别在两种真实情况下，固定随机种子为2，生成20组掷骰子的数据，每组数据包含将2枚骰子一起投掷20次的结果，并在每种真实情况下采用leave-one-out cross-validation，比较上述H1和H2对应模型的结果。

定义 $$CV=-\sum_i\ln(f(y^i_{val}|\theta^{-i}_{cal}))$$
其中$f(.)$为likelihood function，$\theta^{-i}_{cal}$为calibration set的参数拟合结果，$y^i_{val}$表示validation set的数据, $i$表示validation set为第$i$组。

## 建立一个H1为基础的模型
并且生成sample size为20和1000的样本

In [2]:
def H1_generate(num,a,b): # num is the number of trials, a and b are the parameters
    ''' 
    Generate the data under the first hypothesis
    
    Args:
        num: the number of trials
        a: probability of Dice 1 getting 1
        b: probability of Dice 2 getting 6
    '''
    Dice1 = []; Dice2 = []; # initialize the two dice
    for t in range(num):
        if np.random.rand() < a:
            Dice1.append(1) # Dice 1 gets 1
        else:
            Dice1.append(np.random.randint(2, 7)) # generate a random number between 2 and 6
        if np.random.rand() < b:
            Dice2.append(6) # Dice 2 gets 6
        else:
            Dice2.append(np.random.randint(1, 6)) # generate a random number between 1 and 5
    return Dice1, Dice2

In [3]:
[a,b] = [1/3,1/3] # set the parameters
np.random.seed(1) # set the seed
[D1_20,D2_20] = H1_generate(20,a,b) # generate the data of 20 trials
np.random.seed(1) # set the seed
[D1_1000,D2_1000] = H1_generate(1000,a,b) # generate the data of 1000 trials

#### 建立Likelihood函数
分别对H1和H2进行nll的函数构建

In [9]:
def nll_H1(params, D1, D2): # a and b are the parameters, D1 and D2 are the data
    '''
    Calculate the negative log likelihood of the first hypothesis
    Args:
        params: a list of parameters
            a: probability of Dice 1 getting 1
            b: probability of Dice 2 getting 6
        D1: data of Dice 1
        D2: data of Dice 2
    '''
    # initialize the negative log likelihood
    ll = 0;
    [a, b] = params # unpack the parameters
    
    # loop over the data
    for t in range(len(D1)):
        if D1[t] == 1 and D2[t] == 6: # if both dice get 1 and 6
            ll += np.log(a*b) 
        elif D1[t] != 1 and D2[t] == 6: # if Dice 1 does not get 1 but Dice 2 gets 6
            ll += np.log((1-a)*b)
        elif D1[t] == 1 and D2[t] != 6: # if Dice 1 gets 1 but Dice 2 does not get 6
            ll += np.log(a*(1-b))
        else: # if both dice do not get 1 and 6
            ll += np.log((1-a)*(1-b)/25)
    return -ll # return the negative log likelihood

def nll_H2(params, D1, D2): # c is the parameter, D1 and D2 are the data
    '''
    Calculate the negative log likelihood under the second hypothesis
    
    Args:
        params: a list of parameters
            c: probability of (Dice1 getting 1 and Dice2 getting 6) or (Dice1 getting 6 and Dice2 getting 1)
        D1: data of Dice 1
        D2: data of Dice 2
'''
    # initialize the negative log likelihood
    ll = 0;
    [c] = params # unpack the parameters
    
    # loop over the data
    for t in range(len(D1)): 
        if D1[t] == 1 and D2[t] == 6: # if both dice get 1 and 6
            ll += np.log(c)
        elif D1[t] == 6 and D2[t] == 1: # if both dice get 6 and 1
            ll += np.log(c)
        else: # if both dice do not get 1 and 6
            ll += np.log((1-2*c)/34)
    return -ll # return the negative log likelihood

        

#### 拟合
通过MLE对函数参数进行拟合，得到opt

#### 模型比较
根据AIC的计算公式：
$$ AIC = -2\ln_{}{L} (\overline{\theta } \mid y) + 2K $$
再对其进行小样本（n = 20）时的矫正：
$$ AIC_{c} = AIC + \frac{2K(K+1)}{n-K-1}  $$

In [10]:
def fit_H1(D1, D2): # D1 and D2 are the data
    
    optimalh1=[];   # initialize the optimal parameters
    LB = [1e-4, 1e-4]; UB = [1, 1]; # set the lower and upper bounds
    args = [D1, D2] # set the arguments
    N_random=2;
    
    for i in range(N_random):
        [a0, b0] = np.random.uniform(1e-4, 1, 2) # generate random initial values
        H1opt = constrNMPy.constrNM(nll_H1, [a0, b0], LB, UB, args=args, full_output=True)
        optimalh1.append(H1opt['xopt'])
    y = list(map(lambda params: nll_H1(params, *args), optimalh1)) # optional: use full output from constrNM
    opth1 = optimalh1[y.index(min(y))]
    
    return opth1 # return the optimal parameters

def fit_H2(D1, D2): # D1 and D2 are the data
    '''
    Fit the second hypothesis
    
    Args:
        D1: data of Dice 1
        D2: data of Dice 2
    '''
    optimalh2=[];   # initialize the optimal parameters
    LB = [1e-4]; UB = [0.5]; # set the lower and upper bounds
    args = [D1, D2] # set the arguments
    N_random=2;
    
    for i in range(N_random):
        [c0] = np.random.uniform(1e-4, 0.5, 1) # generate random initial values
        H2opt = constrNMPy.constrNM(nll_H2, [c0], LB, UB, args=args, full_output=True)
        optimalh2.append(H2opt['xopt'])
    y = list(map(lambda params: nll_H2(params, *args), optimalh2)) # optional: use full output from constrNM
    opth2 = optimalh2[y.index(min(y))]
    

    return opth2 # return the maximum likelihood estimate of the parameters



In [11]:
print('The model estimations under H1: \n') 

opth1_20 = fit_H1(D1_20, D2_20) # fit the model to the data of 20 trials
print('For ModelH1: MLE[a, b] = ' + str(opth1_20) + ', When n = 20')
# Caculate the AIC of ModelH1
AIC_H1 = 2*nll_H1(opth1_20, D1_20, D2_20) + 2*2 # 2 is the number of parameters
AIC_H1_c = AIC_H1 + 2*2*(2+1)/(20-2-1) # 20 is the number of trials, 2 is the number of parameters
print('For ModelH1 with sample size of 20: '+'AIC = ' + str(AIC_H1) + ', AIC_corrected = ' + str(AIC_H1_c))

opth1_1000 = fit_H1(D1_1000, D2_1000) # fit the model to the data of 1000 trials
print('For ModelH1: MLE[a, b] = ' + str(opth1_1000) + ', When n = 1000')
# Caculate the AIC of ModelH1
AIC_H1 = 2*nll_H1(opth1_1000, D1_1000, D2_1000) # no correction for big sample
print('For ModelH1 with sample size of 1000: '+'AIC = ' + str(AIC_H1))

print('\n \nThe model estimations under H2: \n') 

opth2 = fit_H2(D1_20, D2_20)
print('For ModelH2: MLE[c] = ' + str(opth2) + ' When n = 20')

# Caculate the AIC of ModelH2
AIC_H2 = 2*nll_H2(opth2,D1_20,D2_20) + 2*1 # 1 is the number of parameters
AIC_H2_c = AIC_H2 + 2*1*(2+1)/(20-1-1) # 20 is the number of trials, 1 is the number of parameters
print('For ModelH2 with sample size of 20: '+'AIC = ' + str(AIC_H2) + ', AIC_corrected = ' + str(AIC_H2_c))

opth2 = fit_H2(D1_1000, D2_1000)
print('For ModelH2: MLE[c] = ' + str(opth2) + ' When n = 1000')

# Caculate the AIC of ModelH2
AIC_H2 = 2*nll_H2(opth2,D1_1000,D2_1000) + 2*1 # 1 is the number of parameters
print('For ModelH2 with sample size of 1000: '+'AIC = ' + str(AIC_H2))


The model estimations under H1: 

For ModelH1: MLE[a, b] = [0.4000017  0.35000856], When n = 20
For ModelH1 with sample size of 20: AIC = 121.19584874580305, AIC_corrected = 121.90173109874422
For ModelH1: MLE[a, b] = [0.38399493 0.33501573], When n = 1000
For ModelH1 with sample size of 1000: AIC = 5188.841568837131

 
The model estimations under H2: 

For ModelH2: MLE[c] = [0.14999751] When n = 20
65.74521647026394
For ModelH2 with sample size of 20: AIC = 133.49043294052788, AIC_corrected = 133.82376627386122
For ModelH2: MLE[c] = [0.06700005] When n = 1000
For ModelH2 with sample size of 1000: AIC = 7083.260702972606


### Q3
分别假设骰子的真实情况符合H1和H2，当真实情况为H1时$a=b=1/3$，当真实情况为H2时$c=1/15$。分别在两种真实情况下，固定随机种子为2，生成20组掷骰子的数据，每组数据包含将2枚骰子一起投掷20次的结果，并在每种真实情况下采用leave-one-out cross-validation，比较上述H1和H2对应模型的结果。

定义 $$CV=-\sum_i\ln(f(y^i_{val}|\theta^{-i}_{cal}))$$
其中$f(.)$为likelihood function，$\theta^{-i}_{cal}$为calibration set的参数拟合结果，$y^i_{val}$表示validation set的数据, $i$表示validation set为第$i$组。

In [13]:
def H2_generate(num, c): # num is the number of trials, c are the parameters
    ''' 
    Generate the data under the second hypothesis
    
    Args:
        num: the number of trials
        c: the probability of (Dice1 getting 1 and Dice2 getting 6) or (Dice1 getting 6 and Dice2 getting 1)
    Returns:
        D1: the result of Dice1
        D2: the result of Dice2
    '''
    Dice1 = []; Dice2 = []; # initialize the two dice
    
    # loop over the data
    for i in range(num):
        roll = np.random.rand() # generate a random number between 0 and 1
        if roll < c: # if both dice get 1 and 6 with probability c
            Dice1.append(1)
            Dice2.append(6)
        elif roll < 2*c: # if both dice get 6 and 1 with probability c
            Dice1.append(6)
            Dice2.append(1)
        else: # if both dice do not get 1 and 6 with probability (1 - 2*c)
            while True:
                d1 = np.random.randint(1, 7)
                d2 = np.random.randint(1, 7)
                if d1 != 1 or d2 != 6:
                    if d1 != 6 or d2 != 1:
                        Dice1.append(d1)
                        Dice2.append(d2)
                        break
            
    return Dice1, Dice2

# ## TEST THE MODEL
# [c] = [1/15]; # set the parameters

# # generate the data
# [H2_D1_20, H2_D2_20] = H2_generate(2000, c)

# # test the model
# opth2 = fit_H2(H2_D1_20, H2_D2_20)
# print('\n For ModelH2: MLE[c] = ' + str(opth2) + ' When n = 20')
    

### Generate the required dataset
分别假设骰子的真实情况符合H1和H2，当真实情况为H1时$a=b=1/3$，当真实情况为H2时$c=1/15$。分别在两种真实情况下，固定随机种子为2，生成20组掷骰子的数据，每组数据包含将2枚骰子一起投掷20次的结果，


In [14]:
def dataset_generate(func, args, trials, blocks): 
    '''
    Generate the dataset as required
    
    Args:
        func: the function to generate the data
        args: the arguments of the function
        trials: the number of trials in each block
        blocks: the number of blocks
    Returns:
        Dataset: the dataset
    '''
    Dataset = []; # initialize the dataset
    
    for bn in range(blocks):
        [D1, D2] = func(trials, *args)
        Dataset.append([D1, D2])
    
    return Dataset

np.random.seed(2) # set the seed

dth1 = dataset_generate(H1_generate, [1/3, 1/3], 20, 20) # generate the dataset under H1

np.random.seed(2) # set the seed
dth2 = dataset_generate(H2_generate, [1/15], 20, 20) # generate the dataset under H2

### 进行交叉验证

In [16]:
def Cross_Validify(func_fit, func_nll, data): # leave_one_out cross validation
    '''
    Cross validation
    
    Args:
        func_fit: the function to fit the model
        func_nll: the function to calculate the negative log likelihood
        data: the data
        k: the number of folds
    Returns:
        CV: the sum of the negative log likelihood of the k folds
    '''
    nll = 0 # initialize the negative log likelihood
    
    for bn in range(len(data)):
        train_data = data[bn] # the data for training
        train_D1 = train_data[0]; train_D2 = train_data[1] # the data for training
        opt = func_fit(train_D1, train_D2) # fit the model to the data for training
        for i in range(len(data)):
            if i != bn:
                test_data = data[i]
                nll += func_nll(opt, test_data[0], test_data[1]) # calculate the negative log likelihood of the data for testing
    
    return nll
        
    
print(Cross_Validify(fit_H1, nll_H1, dth1)) # cross validation for H1
print(Cross_Validify(fit_H2, nll_H2, dth2)) # cross validation for H2
print(Cross_Validify(fit_H1, nll_H1, dth2)) # cross validation for H1 under H2
print(Cross_Validify(fit_H2, nll_H2, dth1)) # cross validation for H2 under H1


20561.44633247924
27948.43088789693
25209.196402304366
27374.571105654315
